## Project 4 : Boolean Decision Tree
* CS 5300 : Artificial Intelligent
* Kyu Cho
* 4/13/16

## Data Description
### Intro
The sinking of the RMS Titanic is one of the most infamous shipwrecks in history.  On April 15, 1912, during her maiden voyage, the Titanic sank after colliding with an iceberg, killing 1502 out of 2224 passengers and crew. This sensational tragedy shocked the international community and led to better safety regulations for ships.

One of the reasons that the shipwreck led to such loss of life was that there were not enough lifeboats for the passengers and crew. Although there was some element of luck involved in surviving the sinking, some groups of people were more likely to survive than others, such as women, children, and the upper-class.

### Variables
Note: This data already has been preprocessed.


**Sex** : Sex (1 = Female; 0 = Male)  
**FamilySize** : Family (1 = Aboard with Family; 0 = Aboard without Family)  
**Child** : Child (age < 16) (1 = Child; 0 = Not Child)  
**Pclass.2** : Passenger class (1 = 2nd Class; 0 = 1st or 3rd Class)  
**Pclass.3** : Passenger class (1 = 3rd Class; 0 = 1st or 2nd Class)  
Note : It does not have value for 1st class because if Pclass.2 = 0 and Pclass.3 = 0 meaning it's Pclass.1.  
**Survive** : Survival (0 = No; 1 = Yes)  

## Implementation
Here is the outline of this program.
1. Build a binary decision tree from scratch.
2. Make predictions using the decision tree.
3. Evaluate the accuracy of the decision tree.

I've implemented following conditions to stop building tree or prunning the tree.
1. **Stopping condition 1:** All data points in a node are from the same class.
2. **Stopping condition 2:** No more features to split on.
3. **Early Stopping condition 1:** Reached maximum depth threshold.
4. **Early Stopping condition 2:** Reached minimum node size threshold.
5. **Early Stopping condition 3:** Reached minimum entropy threshold.

In [26]:
import pandas as pd
import numpy as np
import math
import os
os.chdir('C:\Users\Kyu\Documents\python\data')

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [27]:
print(train.head(10))

   Sex  FamilySize  Child  Pclass.2  Pclass.3  Survived
0    0           0      0         0         1         0
1    1           0      0         0         0         1
2    1           0      0         0         0         1
3    0           0      0         0         1         0
4    0           0      0         0         0         0
5    0           1      1         0         1         0
6    1           1      0         0         1         1
7    1           0      1         1         0         1
8    1           1      1         0         1         1
9    1           0      0         0         0         1


In [28]:
print(test.head(10))

   Sex  FamilySize  Child  Pclass.2  Pclass.3  Survived
0    1           0      0         0         1         1
1    0           0      0         0         1         0
2    0           0      0         0         1         0
3    0           0      0         1         0         1
4    0           0      0         0         0         1
5    0           0      0         1         0         0
6    0           0      0         0         0         0
7    0           0      0         0         1         0
8    0           0      0         0         1         0
9    1           0      0         1         0         1


In [29]:
features = ['Sex',
            'FamilySize',
            'Child',
            'Pclass.2',
            'Pclass.3']
target = 'Survived'

In [30]:
# Entropy and information gain calculation function
def getEntropy(data, feature, target):
    tblSex = pd.crosstab(data[feature], data[target])
    
    if tblSex.shape[0] == 2:
        mtrixArr = np.array(tblSex/tblSex.sum(axis = 1))
        p1 = mtrixArr[0,0]
        p2 = 1 - p1
        p3 = mtrixArr[1,1]
        p4 = 1- p3 
        
        mtrixArr2 = np.array(tblSex.sum(axis = 1)/len(train))
        pf1 = mtrixArr2[0]
        pf2 = mtrixArr2[1]
        
        # Entropy formula
        if p1 == 0 or p2 == 0 or p3 == 0 or p4 == 0:
            return 0
        else:
            entropy = pf1*(-p1*math.log(p1) - p2*math.log(p2)) + pf2*(-p3*math.log(p3) - p4*math.log(p4))
            return entropy
    else:
            return 0

In [31]:
# Selecting best feature to split next, based on the entropy
def best_splitting_feature(data, features, target):
    best_feature = None
    best_infoGain = 0;
    entropy = 0
    for feature in features:
        # Calculate the information gain in the left split
        entropy = getEntropy(data, feature, target)
        infoGain = 1 - entropy
        if infoGain == 1:
            print ("No more informaion gain")
            return None, 0
            
        if infoGain > best_infoGain:
            best_infoGain = infoGain
            best_feature = feature
    print ("Feature : [%s] Entropy: %s" % (best_feature, round(entropy, 2)))
    return best_feature, entropy

In [32]:
# Ceating empty node function
def create_leaf(target_values):    
    # Create a leaf node
    leaf = {'splitting_feature' : None,
            'left' : None,
            'right' : None,
            'is_leaf': True}  
    
    # Count the number of data points that are +1 and -1 in this node.
    num_ones = len(target_values[target_values == +1])
    num_minus_ones = len(target_values[target_values == 0])
    
    # For the leaf node, set the prediction to be the majority class.
    # Store the predicted class (1 or -1) in leaf['prediction']
    if num_ones > num_minus_ones:
        leaf['prediction'] = +1    
    else:
        leaf['prediction'] = 0      
        
    # Return the leaf node        
    return leaf 

In [42]:
# Model building function
def decision_tree_create(data, features, target, current_depth = 0, 
                         max_depth = 10, min_node_size=1, 
                         min_entropy=0.0):
    remaining_features = features[:] # Make a copy of the features.
    
    entropy = 0
    target_values = data[target]
    print("--------------------------------------------------------------------")
    print("Depth = %s (%s data points)" % (current_depth, len(target_values)))
    
    # Stopping condition 2: No more features to split on.
    if remaining_features == []:  
        print("Stopping condition 2. \n\tNo remaining features.")     
        # If there are no remaining features to consider, make current node a leaf node
        return create_leaf(target_values) 
    
    # Early stopping condition 1: Reached max depth limit.
    if current_depth >= max_depth:  
        print("Early stopping condition 1. \n\tReached maximum depth.")
        # If the max tree depth has been reached, make current node a leaf node
        return create_leaf(target_values)

    # Early stopping condition 2: Data has less or equal to the minimum size.
    if len(data) <= min_node_size:
        print("Early stopping condition 2. \n\tReached minimum node size.")
        return create_leaf(target_values) 
    
    # Find the best splitting feature (recall the function best_splitting_feature implemented above)
    splitting_feature, entropy = best_splitting_feature(data, features, target)  
    
    # Stopping condition 1: All nodes are of the same type.
    if splitting_feature == None:
        print("Stopping condition 1. \n\tAll data points have the same target value.")   
        #  All data points in a node are from the same class.
        return create_leaf(target_values)
        
    # Early stopping condition 3: Entropy value is less or equal to the minimum threshold.
    if entropy <= min_entropy:
        print("Early stopping condition 3. \n\tMinimum entropy.")
        return create_leaf(target_values)  
    
    # Split on the best feature that we found. 
    left_split = data[data[splitting_feature] == 0]
    right_split = data[data[splitting_feature] == 1]   
    remaining_features.remove(splitting_feature)
    print("Split on feature [%s] (%s, %s)" % (\
                    splitting_feature, len(left_split), len(right_split)))
    
        
    # Create a leaf node if the split is "perfect"
    if len(left_split) == len(data):
        print("Creating leaf node.")
        return create_leaf(left_split[target])
    if len(right_split) == len(data):
        print("Creating leaf node.")
        return create_leaf(right_split[target])
        
    # Repeat (recurse) on left and right subtrees
    left_tree = decision_tree_create(left_split, remaining_features, target, 
                                     current_depth + 1, max_depth, min_node_size, min_entropy)        
    
    right_tree = decision_tree_create(right_split, remaining_features, target, 
                                     current_depth + 1, max_depth, min_node_size, min_entropy)
    
    return {'is_leaf'          : False, 
            'prediction'       : None,
            'splitting_feature': splitting_feature,
            'left'             : left_tree, 
            'right'            : right_tree}

In [43]:
# Counting node function
def count_nodes(tree):
    if tree['is_leaf']:
        return 1
    return 1 + count_nodes(tree['left']) + count_nodes(tree['right'])

In [44]:
# Traversing Tree function for the query 
def classify(tree, x):   
    # if the node is a leaf node.
    if tree['is_leaf']:
        return tree['prediction'] 
    else:
        # split on feature.
        split_feature_value = x[tree['splitting_feature']]
        split_feature_value = np.array(split_feature_value)[0]
        if split_feature_value == 0:
            return classify(tree['left'], x)
        else:
            return classify(tree['right'], x) 

In [45]:
# Predicting output function
def predict(model, data):
    prediction = []
    for i in range(0, len(data)):
        pred = classify(model, data[i:i+1])
        prediction.append(pred)
    return prediction

In [46]:
# Calculating accuracy function            
def getAccuracy(prediction, data):
    mask = (prediction == data["Survived"])
    return round(float(len(mask[mask == True])) / len(data), 4)

In [47]:
print("--------------------------------------------------------------------")
print("\tBuilding Tree Start")
model = decision_tree_create(train, features, 'Survived', max_depth = 5, 
                                min_node_size = 4, min_entropy=.03)

--------------------------------------------------------------------
	Building Tree Start
--------------------------------------------------------------------
Depth = 0 (712 data points)
Feature : [Sex] Entropy: 0.62
Split on feature [Sex] (463, 249)
--------------------------------------------------------------------
Depth = 1 (463 data points)
Feature : [Child] Entropy: 0.32
Split on feature [Child] (417, 46)
--------------------------------------------------------------------
Depth = 2 (417 data points)
Feature : [Pclass.2] Entropy: 0.27
Split on feature [Pclass.2] (339, 78)
--------------------------------------------------------------------
Depth = 3 (339 data points)
Feature : [Pclass.3] Entropy: 0.22
Split on feature [Pclass.3] (97, 242)
--------------------------------------------------------------------
Depth = 4 (97 data points)
Feature : [FamilySize] Entropy: 0.09
Split on feature [FamilySize] (85, 12)
--------------------------------------------------------------------
Dept

In [48]:
print("--------------------------------------------------------------------")
print("Total number of node : %s" % count_nodes(model))

--------------------------------------------------------------------
Total number of node : 19


In [49]:
prediction = predict(model, test)
print("Prediction :", prediction)

('Prediction :', [1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0])


In [50]:
accuracy = getAccuracy(prediction, test)
print("Accuracy :" , accuracy)

('Accuracy :', 0.8362)
